### Libraries

In [2]:
rm(list=ls())
# dependencies
Sys.setenv(LANG = "en")
Sys.setenv(JAVA_HOME = "C:\\Program Files\\Java\\jre1.8.0_261")
library(readr)
library(ggplot2)
library(ggcorrplot)
library(dplyr)
library(dummies)
library(fastDummies)
library(olsrr)
library(rJava)
library(glmulti)
library(stargazer)
library(car)
library("lmtest")
library("tseries")
library("sandwich")



library("MASS")
library("sandwich")
library("zoo")
library("car")
library("lmtest")
library("Formula")
library("plm")
library("stargazer")
options(warn=-1)

### Loading data

In [3]:
df <- read_csv("../../data/robustness_check/processed_data/final_dataset_all.csv")

Parsed with column specification:
cols(
  .default = col_double(),
  country_name = col_character(),
  country_text_id = col_character(),
  continent = col_character(),
  sub_region = col_character(),
  income_group = col_character()
)
See spec(...) for full column specifications.


In [4]:
names(df)

[1] "country_name"    "country_text_id" "continent"       "sub_region"     
 [5] "region_pol"      "region_geo"      "income_group"    "pandem_old"     
 [9] "pandem_dis_old"  "panback_old"     "rule"            "polyarchy"      
[13] "education"       "gini"            "density"         "gdp_pc"         
[17] "inflation"       "oil"             "mineral"         "trade_gdp"      
[21] "ethnic_frac"     "ling_frac"       "relig_frac"      "pandem"         
[25] "panback"         "cases_ratio"

In [5]:
# Dataset preparation
## Removing redundant variables
df$country_text_id <- NULL
df$continent <- NULL
df$sub_region <- NULL
df$region_pol <- NULL
df$pandem_old <- NULL
df$pandem_dis_old <- NULL
df$panback_old <- NULL

In [6]:
## Dummies
data <- fastDummies::dummy_cols(df,select_columns = c("income_group","region_geo"),remove_most_frequent_dummy = TRUE, remove_selected_columns = TRUE)

In [7]:
data <- data %>% rename(
    low_income = 'income_group_Low income',
    lower_middle_income = 'income_group_Lower middle income',
    upper_middle_income = 'income_group_Upper middle income'
    )

In [8]:
## Simple feature eng.
data$gini_log <- log(data$gini) 
data$density_log <- log(data$density) 
data$gdp_pc_log <- log(data$gdp_pc) 
data$trade_gdp_log <- log(data$trade_gdp)
data$education_log <- log(data$education) 

data$rule2 <- data$rule * data$rule
data$polyarchy2 <- data$polyarchy * data$polyarchy

colnames(data)

[1] "country_name"        "rule"                "polyarchy"          
 [4] "education"           "gini"                "density"            
 [7] "gdp_pc"              "inflation"           "oil"                
[10] "mineral"             "trade_gdp"           "ethnic_frac"        
[13] "ling_frac"           "relig_frac"          "pandem"             
[16] "panback"             "cases_ratio"         "low_income"         
[19] "lower_middle_income" "upper_middle_income" "region_geo_10"      
[22] "region_geo_11"       "region_geo_12"       "region_geo_13"      
[25] "region_geo_14"       "region_geo_15"       "region_geo_16"      
[28] "region_geo_17"       "region_geo_18"       "region_geo_19"      
[31] "region_geo_2"        "region_geo_3"        "region_geo_4"       
[34] "region_geo_5"        "region_geo_6"        "region_geo_7"       
[37] "region_geo_8"        "region_geo_9"        "gini_log"           
[40] "density_log"         "gdp_pc_log"          "trade_gdp_log"      
[43] "education_log"       "rule2"               "polyarchy2"

In [9]:
head(data)

country_name,rule,polyarchy,education,gini,density,gdp_pc,inflation,oil,mineral,...,region_geo_7,region_geo_8,region_geo_9,gini_log,density_log,gdp_pc_log,trade_gdp_log,education_log,rule2,polyarchy2
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Afghanistan,0.341,0.347,3.813000,27.8,56.93776,502.1155,2.302373,0.001254688,0.007420852,...,0,0,0,3.325036,4.041959,6.218830,3.820372,1.338416,0.116281,0.120409
Albania,0.596,0.481,8.457939,33.2,104.61226,5352.8574,1.411091,1.365752443,0.146426288,...,0,0,0,3.502550,4.650261,8.585386,4.340271,2.135106,0.355216,0.231361
Algeria,0.217,0.300,7.300000,27.6,17.73008,3948.3433,1.951768,12.313696685,0.046716900,...,0,0,0,3.317816,2.875262,8.281051,3.952104,1.987874,0.047089,0.090000
Angola,0.406,0.386,2.980000,51.3,24.71305,2973.5912,17.145320,15.753913421,0.000000000,...,1,0,0,3.937691,3.207332,7.997526,4.195366,1.091923,0.164836,0.148996
Argentina,0.757,0.812,8.770000,41.4,16.25851,10006.1490,53.548304,0.735824027,0.308309752,...,0,0,0,3.723281,2.788616,9.210955,3.477963,2.171337,0.573049,0.659344
Armenia,0.800,0.805,10.818000,34.4,103.68022,4622.7335,1.443447,0.000000000,5.590611662,...,0,0,0,3.538057,4.641311,8.438741,4.515599,2.381211,0.640000,0.648025


In [10]:
# robust_pandem_dis_2 = coeftest(model_pandem_dis_2, vcov=vcovHC(model_pandem_dis_2, type="HC0"))
# show(robust_pandem_dis_2)

# model_pandem_dis_3 <- lm(pandem_dis ~ polyarchy + education + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
# olsrr::ols_coll_diag(model_pandem_dis_3)
# olsrr::ols_test_normality(model_pandem_dis_3)
# resettest(model_pandem_dis_3, power=2:3, type="fitted")
# bptest(model_pandem_dis_3, studentize=TRUE)
# dwtest(model_pandem_dis_3)




# stargazer(model_panback_1, model_pandem_1, type="html", out="wo_military/all_fe.html")
# stargazer(model_panback_2, model_pandem_2, type="html", out="wo_military/all_rule_fe.html")
# stargazer(model_panback_2, model_pandem_2, type="html", out="wo_military/all_rule_fe_robust.html")
# stargazer(model_panback_3, model_pandem_3, type="html", out="wo_military/all_poly_fe.html")

### Function for diagnostic tests

In [11]:
diagnostic_tests <- function(model) {
    
    cat("VIF for collinearity of variables:\n\n")
    print(olsrr::ols_coll_diag(model)[1]$vif_t)
    
    cat("\n\n\nKolmogorov-Smirnov for normality:\n")
    print(olsrr::ols_test_normality(model)[1]$kolmogorv)
    
    cat("\n\n\nRESET test for correct specification:\n")
    print(resettest(model, power=2:3, type="fitted"))
    
    cat("\n\n\nBreusch-Pagan test for homogeneity:\n")
    print(bptest(model, studentize=TRUE))
    
    cat("\n\n\nDurbin-Watson test for autocorrelation:\n")
    print(dwtest(model))
}

### Main tables

In [12]:
### PANDEM

model_pandem_1 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_2 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_3 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_4 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_5 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_6 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)



### PANBACK

model_panback_1 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_2 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_3 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_4 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_5 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_6 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)

In [13]:
diagnostic_tests(model_pandem_1)

VIF for collinearity of variables:

             Variables  Tolerance       VIF
1                 rule 0.14893762  6.714220
2            polyarchy 0.15921285  6.280900
3          cases_ratio 0.43293097  2.309837
4        education_log 0.21510908  4.648804
5          ethnic_frac 0.27103537  3.689555
6            ling_frac 0.25608601  3.904938
7           relig_frac 0.41897846  2.386758
8           gdp_pc_log 0.04571730 21.873560
9             gini_log 0.35521540  2.815193
10           inflation 0.67238235  1.487249
11       trade_gdp_log 0.50820509  1.967710
12                 oil 0.44614569  2.241420
13             mineral 0.52479949  1.905490
14         density_log 0.36911959  2.709149
15       region_geo_10 0.18347977  5.450192
16        region_geo_2 0.40243038  2.484902
17        region_geo_3 0.31514942  3.173098
18        region_geo_4 0.27268485  3.667237
19        region_geo_5 0.30051111  3.327664
20        region_geo_6 0.10609859  9.425196
21        region_geo_7 0.16750478  5.969

#### Robust coefficient in case of not fulfilment Breusch-Pagan test

In [14]:
robust_panback_1 = coeftest(model_panback_1, vcov=vcovHC(model_panback_1, type="HC0"))
robust_panback_2 = coeftest(model_panback_2, vcov=vcovHC(model_panback_2, type="HC0"))
robust_panback_3 = coeftest(model_panback_3, vcov=vcovHC(model_panback_3, type="HC0"))
robust_panback_4 = coeftest(model_panback_4, vcov=vcovHC(model_panback_4, type="HC0"))

### Table 1 - regressions with categorical variables

In [15]:
stargazer(model_pandem_1, model_pandem_2, model_pandem_3, type="text")
stargazer(robust_panback_1, robust_panback_2, robust_panback_3, type="text")          


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.021                 -0.149**                                
                            (0.088)                 (0.063)                                
                                                                                           
polyarchy                  -0.216**                                        -0.233***       
                            (0.104)                                         (0.074)        
                                                                               


                         Dependent variable:     
                    -----------------------------
                                                 
                       (1)       (2)       (3)   
-------------------------------------------------
rule                0.539***  0.734***           
                     (0.127)   (0.122)           
                                                 
rule2               -0.506*** -0.676***          
                     (0.128)   (0.122)           
                                                 
polyarchy           0.702***            0.977*** 
                     (0.156)             (0.159) 
                                                 
polyarchy2          -0.696***           -0.965***
                     (0.162)             (0.162) 
                                                 
cases_ratio          0.00000   0.00000  -0.00000 
                    (0.00001) (0.00001) (0.00001)
                                                 

### Table 2 - regressions without categorical variables

In [16]:
stargazer(model_pandem_4, model_pandem_5, model_pandem_6, type="text")
stargazer(robust_panback_4, model_panback_5, model_panback_6, type="text")


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.064                 -0.199***                               
                            (0.072)                 (0.055)                                
                                                                                           
polyarchy                  -0.221***                                       -0.268***       
                            (0.078)                                         (0.058)        
                                                                               

### Saving main tables (replicated regressions according to previous research with updated dependent variables)

In [17]:
stargazer(robust_panback_1, model_pandem_1, robust_panback_2, model_pandem_2,
          robust_panback_3, model_pandem_3,
          type="html", out="../../results/robustness_check/Table_1_regression.html")

stargazer(robust_panback_4, model_pandem_4, model_panback_5, model_pandem_5,
          model_panback_6, model_pandem_6, 
          type="html", out="../../results/robustness_check/Table_2_regression.html")


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td></td><td>pandem</td><td></td><td>pandem</td><td></td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td><em>coefficient</em></td><td><em>OLS</em></td><td><em>coefficient</em></td><td><em>OLS</em></td><td><em>coefficient</em></td><td><em>OLS</em></td></tr>
<tr><td style="text-align:left"></td><td><em>test</em></td><td><em></em></td><td><em>test</em></td><td><em></em></td><td><em>test</em></td><td><em></em></td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.539<sup>***</sup></td><t


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td></td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td><em>coefficient</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td></tr>
<tr><td style="text-align:left"></td><td><em>test</em></td><td><em></em></td><td><em></em></td><td><em></em></td><td><em></em></td><td><em></em></td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.475<sup>***</sup></td><td>-0.064</

# Part with robustness check

## 1) Time of covid - Q2/Q3/Q4 of 2020

### Q2

In [18]:
df <- read_csv("../../data/robustness_check/processed_data/final_dataset_Q2.csv")
df$country_text_id <- NULL
df$continent <- NULL
df$sub_region <- NULL
df$region_pol <- NULL
df$pandem_old <- NULL
df$pandem_dis_old <- NULL
df$panback_old <- NULL
data_Q2 <- fastDummies::dummy_cols(df,select_columns = c("income_group","region_geo"),remove_most_frequent_dummy = TRUE, remove_selected_columns = TRUE)
data_Q2 <- data_Q2 %>% rename(
    low_income = 'income_group_Low income',
    lower_middle_income = 'income_group_Lower middle income',
    upper_middle_income = 'income_group_Upper middle income'
    )
data_Q2$gini_log <- log(data_Q2$gini) 
data_Q2$density_log <- log(data_Q2$density) 
data_Q2$gdp_pc_log <- log(data_Q2$gdp_pc) 
data_Q2$trade_gdp_log <- log(data_Q2$trade_gdp)
data_Q2$education_log <- log(data_Q2$education) 

data_Q2$rule2 <- data_Q2$rule * data_Q2$rule
data_Q2$polyarchy2 <- data_Q2$polyarchy * data_Q2$polyarchy

Parsed with column specification:
cols(
  .default = col_double(),
  country_name = col_character(),
  country_text_id = col_character(),
  continent = col_character(),
  sub_region = col_character(),
  income_group = col_character()
)
See spec(...) for full column specifications.


In [19]:
### CASES PANDEM

model_pandem_1_Q2 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_pandem_2_Q2 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_pandem_3_Q2 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_pandem_4_Q2 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)
model_pandem_5_Q2 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)
model_pandem_6_Q2 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)



### CASES PANBACK

model_panback_1_Q2 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_panback_2_Q2 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_panback_3_Q2 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_panback_4_Q2 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)
model_panback_5_Q2 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)
model_panback_6_Q2 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)

### Table 3 - Q2

In [20]:
stargazer(model_pandem_1_Q2, model_pandem_2_Q2, model_pandem_3_Q2, type="text")
stargazer(model_panback_1_Q2, model_panback_2_Q2, model_panback_3_Q2, type="text")
stargazer(model_pandem_4_Q2, model_pandem_5_Q2, model_pandem_6_Q2, type="text")
stargazer(model_panback_4_Q2, model_panback_5_Q2, model_panback_6_Q2, type="text")


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.045                 -0.145**                                
                            (0.087)                 (0.062)                                
                                                                                           
polyarchy                   -0.167                                         -0.205***       
                            (0.103)                                         (0.072)        
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    panback                                
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                       0.678***                0.868***                                
                            (0.177)                 (0.169)                                
                                                                                           
rule2                      -0.666***               -0.781***                               
                            (0.178)                 (0.167)                                
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.072                 -0.200***                               
                            (0.072)                 (0.054)                                
                                                                                           
polyarchy                  -0.206***                                       -0.259***       
                            (0.078)                                         (0.058)        
                                                                               

# Q3

In [21]:
df <- read_csv("../../data/robustness_check/processed_data/final_dataset_Q3.csv")
df$country_text_id <- NULL
df$continent <- NULL
df$sub_region <- NULL
df$region_pol <- NULL
df$pandem_old <- NULL
df$pandem_dis_old <- NULL
df$panback_old <- NULL
data_Q3 <- fastDummies::dummy_cols(df,select_columns = c("income_group","region_geo"),remove_most_frequent_dummy = TRUE, remove_selected_columns = TRUE)
data_Q3 <- data_Q3 %>% rename(
    low_income = 'income_group_Low income',
    lower_middle_income = 'income_group_Lower middle income',
    upper_middle_income = 'income_group_Upper middle income'
    )
data_Q3$gini_log <- log(data_Q3$gini) 
data_Q3$density_log <- log(data_Q3$density) 
data_Q3$gdp_pc_log <- log(data_Q3$gdp_pc) 
data_Q3$trade_gdp_log <- log(data_Q3$trade_gdp)
data_Q3$education_log <- log(data_Q3$education) 

data_Q3$rule2 <- data_Q3$rule * data_Q3$rule
data_Q3$polyarchy2 <- data_Q3$polyarchy * data_Q3$polyarchy


Parsed with column specification:
cols(
  .default = col_double(),
  country_name = col_character(),
  country_text_id = col_character(),
  continent = col_character(),
  sub_region = col_character(),
  income_group = col_character()
)
See spec(...) for full column specifications.


In [22]:
### CASES PANDEM

model_pandem_1_Q3 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_pandem_2_Q3 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_pandem_3_Q3 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_pandem_4_Q3 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)
model_pandem_5_Q3 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)
model_pandem_6_Q3 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)



### CASES PANBACK

model_panback_1_Q3 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_panback_2_Q3 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_panback_3_Q3 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_panback_4_Q3 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)
model_panback_5_Q3 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)
model_panback_6_Q3 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)

### Table 4 - Q3

In [23]:
stargazer(model_pandem_1_Q3, model_pandem_2_Q3, model_pandem_3_Q3, type="text")
stargazer(model_panback_1_Q3, model_panback_2_Q3, model_panback_3_Q3, type="text")
stargazer(model_pandem_4_Q3, model_pandem_5_Q3, model_pandem_6_Q3, type="text")
stargazer(model_panback_4_Q3, model_panback_5_Q3, model_panback_6_Q3, type="text")


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.026                 -0.148**                                
                            (0.084)                 (0.061)                                
                                                                                           
polyarchy                  -0.212**                                        -0.233***       
                            (0.102)                                         (0.073)        
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    panback                                
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                       0.508***                0.714***                                
                            (0.175)                 (0.168)                                
                                                                                           
rule2                      -0.486***               -0.663***                               
                            (0.175)                 (0.166)                                
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.070                 -0.228***                               
                            (0.068)                 (0.052)                                
                                                                                           
polyarchy                  -0.259***                                       -0.311***       
                            (0.075)                                         (0.055)        
                                                                               

### Q4

In [24]:
df <- read_csv("../../data/robustness_check/processed_data/final_dataset_Q2.csv")
df$country_text_id <- NULL
df$continent <- NULL
df$sub_region <- NULL
df$region_pol <- NULL
df$pandem_old <- NULL
df$pandem_dis_old <- NULL
df$panback_old <- NULL
data_Q4 <- fastDummies::dummy_cols(df,select_columns = c("income_group","region_geo"),remove_most_frequent_dummy = TRUE, remove_selected_columns = TRUE)
data_Q4 <- data_Q4 %>% rename(
    low_income = 'income_group_Low income',
    lower_middle_income = 'income_group_Lower middle income',
    upper_middle_income = 'income_group_Upper middle income'
    )
data_Q4$gini_log <- log(data_Q4$gini) 
data_Q4$density_log <- log(data_Q4$density) 
data_Q4$gdp_pc_log <- log(data_Q4$gdp_pc) 
data_Q4$trade_gdp_log <- log(data_Q4$trade_gdp)
data_Q4$education_log <- log(data_Q4$education) 

data_Q4$rule2 <- data_Q4$rule * data_Q4$rule
data_Q4$polyarchy2 <- data_Q4$polyarchy * data_Q4$polyarchy

Parsed with column specification:
cols(
  .default = col_double(),
  country_name = col_character(),
  country_text_id = col_character(),
  continent = col_character(),
  sub_region = col_character(),
  income_group = col_character()
)
See spec(...) for full column specifications.


In [25]:
### CASES PANDEM

model_pandem_1_Q4 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_pandem_2_Q4 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_pandem_3_Q4 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_pandem_4_Q4 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)
model_pandem_5_Q4 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)
model_pandem_6_Q4 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)



### CASES PANBACK

model_panback_1_Q4 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_panback_2_Q4 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_panback_3_Q4 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_panback_4_Q4 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)
model_panback_5_Q4 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)
model_panback_6_Q4 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)

### Table 5 - Q4

In [26]:
stargazer(model_pandem_1_Q4, model_pandem_2_Q4, model_pandem_3_Q4, type="text")
stargazer(model_panback_1_Q4, model_panback_2_Q4, model_panback_3_Q4, type="text")
stargazer(model_pandem_4_Q4, model_pandem_5_Q4, model_pandem_6_Q4, type="text")
stargazer(model_panback_4_Q4, model_panback_5_Q4, model_panback_6_Q4, type="text")


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.045                 -0.145**                                
                            (0.087)                 (0.062)                                
                                                                                           
polyarchy                   -0.167                                         -0.205***       
                            (0.103)                                         (0.072)        
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    panback                                
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                       0.678***                0.868***                                
                            (0.177)                 (0.169)                                
                                                                                           
rule2                      -0.666***               -0.781***                               
                            (0.178)                 (0.167)                                
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.072                 -0.200***                               
                            (0.072)                 (0.054)                                
                                                                                           
polyarchy                  -0.206***                                       -0.259***       
                            (0.078)                                         (0.058)        
                                                                               

## Summary

In [27]:
stargazer(model_pandem_1, model_pandem_1_Q2, model_pandem_1_Q3, model_pandem_1_Q4,
          model_pandem_2, model_pandem_2_Q2, model_pandem_2_Q3, model_pandem_2_Q4,
          model_pandem_3, model_pandem_3_Q2, model_pandem_3_Q3, model_pandem_3_Q4,
          type="html", out="../../results/robustness_check/robustness_TIME_pandem_FE.html")

stargazer(model_pandem_4, model_pandem_4_Q2, model_pandem_4_Q3, model_pandem_4_Q4,
          model_pandem_5, model_pandem_5_Q2, model_pandem_5_Q3, model_pandem_5_Q4,
          model_pandem_6, model_pandem_6_Q2, model_pandem_6_Q3, model_pandem_6_Q4,
          type="html", out="../../results/robustness_check/robustness_TIME_pandem_wFE.html")

stargazer(model_panback_1, model_panback_1_Q2, model_panback_1_Q3, model_panback_1_Q4,
          model_panback_2, model_panback_2_Q2, model_panback_2_Q3, model_panback_2_Q4,
          model_panback_3, model_panback_3_Q2, model_panback_3_Q3, model_panback_3_Q4,
          type="html", out="../../results/robustness_check/robustness_TIME_panback_FE.html")

stargazer(model_panback_4, model_panback_4_Q2, model_panback_4_Q3, model_panback_4_Q4,
          model_panback_5, model_panback_5_Q2, model_panback_5_Q3, model_panback_5_Q4,
          model_panback_6, model_panback_6_Q2, model_panback_6_Q3, model_panback_6_Q4,
          type="html", out="../../results/robustness_check/robustness_TIME_panback_wFE.html")


<table style="text-align:center"><tr><td colspan="13" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="12"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="12" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="12">pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td><td>(7)</td><td>(8)</td><td>(9)</td><td>(10)</td><td>(11)</td><td>(12)</td></tr>
<tr><td colspan="13" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>-0.021</td><td>-0.045</td><td>-0.026</td><td>-0.045</td><td>-0.149<sup>**</sup></td><td>-0.145<sup>**</sup></td><td>-0.148<sup>**</sup></td><td>-0.145<sup>**</sup></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.088)</td><td>(0.087)</td><td>(0.084)</td><td>(0.087)</td><td>(0.063)</td><td>(0.062)</td><td>


<table style="text-align:center"><tr><td colspan="13" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="12"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="12" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="12">pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td><td>(7)</td><td>(8)</td><td>(9)</td><td>(10)</td><td>(11)</td><td>(12)</td></tr>
<tr><td colspan="13" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>-0.064</td><td>-0.072</td><td>-0.070</td><td>-0.072</td><td>-0.199<sup>***</sup></td><td>-0.200<sup>***</sup></td><td>-0.228<sup>***</sup></td><td>-0.200<sup>***</sup></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.072)</td><td>(0.072)</td><td>(0.068)</td><td>(0.072)</td><td>(0.055)</td><td>(0.054)</td>


<table style="text-align:center"><tr><td colspan="13" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="12"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="12" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="12">panback</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td><td>(7)</td><td>(8)</td><td>(9)</td><td>(10)</td><td>(11)</td><td>(12)</td></tr>
<tr><td colspan="13" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.539<sup>***</sup></td><td>0.678<sup>***</sup></td><td>0.508<sup>***</sup></td><td>0.678<sup>***</sup></td><td>0.734<sup>***</sup></td><td>0.868<sup>***</sup></td><td>0.714<sup>***</sup></td><td>0.868<sup>***</sup></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.165)</td><td>(0.177)</td><td>(0.175)</td


<table style="text-align:center"><tr><td colspan="13" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="12"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="12" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="12">panback</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td><td>(7)</td><td>(8)</td><td>(9)</td><td>(10)</td><td>(11)</td><td>(12)</td></tr>
<tr><td colspan="13" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.475<sup>***</sup></td><td>0.654<sup>***</sup></td><td>0.456<sup>***</sup></td><td>0.654<sup>***</sup></td><td>0.734<sup>***</sup></td><td>0.927<sup>***</sup></td><td>0.710<sup>***</sup></td><td>0.927<sup>***</sup></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.140)</td><td>(0.153)</td><td>(0.151)</td

## 2) COLINEARITY
- remove gini_log and income categories because collinear with gdp_pc
- fractionalization instead of 3 fracs

In [28]:
data$fractionalization <- data$ethnic_frac + data$ling_frac + data$relig_frac

In [29]:
### CASES PANDEM

model_pandem_1_col <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_pandem_2_col <- lm(pandem ~ rule + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_pandem_3_col <- lm(pandem ~ polyarchy + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_pandem_4_col <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_5_col <- lm(pandem ~ rule + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_6_col <- lm(pandem ~ polyarchy + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)



### CASES PANBACK

model_panback_1_col <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_panback_2_col <- lm(panback ~ rule + rule2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_panback_3_col <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_panback_4_col <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_5_col <- lm(panback ~ rule + rule2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_6_col <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)


In [30]:
diagnostic_tests(model_pandem_4_col)

VIF for collinearity of variables:

           Variables Tolerance      VIF
1               rule 0.2314155 4.321233
2          polyarchy 0.2916691 3.428543
3        cases_ratio 0.7076848 1.413058
4      education_log 0.4142991 2.413715
5  fractionalization 0.7238791 1.381446
6         gdp_pc_log 0.2115039 4.728046
7          inflation 0.8517730 1.174022
8      trade_gdp_log 0.8563672 1.167723
9                oil 0.6404905 1.561303
10           mineral 0.7407231 1.350032
11       density_log 0.7557013 1.323274



Kolmogorov-Smirnov for normality:

	One-sample Kolmogorov-Smirnov test

data:  y
D = 0.11442, p-value = 0.04731
alternative hypothesis: two-sided




RESET test for correct specification:

	RESET test

data:  model
RESET = 9.329, df1 = 2, df2 = 129, p-value = 0.0001644




Breusch-Pagan test for homogeneity:

	studentized Breusch-Pagan test

data:  model
BP = 6.22, df = 11, p-value = 0.8583




Durbin-Watson test for autocorrelation:

	Durbin-Watson test

data:  model
DW = 1.9

In [31]:
robust_panback_3_col = coeftest(model_panback_3_col, vcov=vcovHC(model_panback_3_col, type="HC0"))

In [32]:
# ## Summary

# stargazer(model_pandem_1, model_pandem_1_col,
#           model_pandem_2, model_pandem_2_col,
#           model_pandem_3, model_pandem_3_col,
#           type="html", out="wo_military/COL_pandem_FE.html")

# stargazer(model_pandem_4, model_pandem_4_col,
#           model_pandem_5, model_pandem_5_col,
#           model_pandem_6, model_pandem_6_col,
#           type="html", out="wo_military/COL_pandem_wFE.html")

# stargazer(model_panback_1, model_panback_1_col,
#           model_panback_2, model_panback_2_col,
#           model_panback_3, robust_panback_3_col,
#           type="html", out="wo_military/COL_panback_FE.html")

# stargazer(model_panback_4, model_panback_4_col,
#           model_panback_5, model_panback_5_col,
#           model_panback_6, model_panback_6_col,
#           type="html", out="wo_military/COL_panback_wFE.html")

In [33]:
stargazer(model_panback_1_col, model_pandem_1_col,  
          model_panback_2_col, model_pandem_2_col, 
          robust_panback_3_col, model_pandem_3_col, 
          type="html", out="../../results/robustness_check/robustness_COLLINEARITY_FE.html")


stargazer(model_panback_4_col, model_pandem_4_col, 
          model_panback_5_col, model_pandem_5_col, 
          model_panback_6_col, model_pandem_6_col, 
          type="html", out="../../results/robustness_check/robustness_COLLINEARITY_wFE.html")


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td></td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>coefficient</em></td><td><em>OLS</em></td></tr>
<tr><td style="text-align:left"></td><td><em></em></td><td><em></em></td><td><em></em></td><td><em></em></td><td><em>test</em></td><td><em></em></td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.538<sup>***</sup></td><td>-0.042</


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.491<sup>***</sup></td><td>-0.060</td><td>0.744<sup>***</sup></td><td>-0.206<sup>***</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.136)</td><td>(0.072)</td><td>(0.122)</td><td>(0.055)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text

### 3) Bez missingów
- remove gini
- remove education

In [34]:
### CASES PANDEM

model_pandem_1_miss <- lm(pandem ~ rule + polyarchy + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_2_miss <- lm(pandem ~ rule + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_3_miss <- lm(pandem ~ polyarchy + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_4_miss <- lm(pandem ~ rule + polyarchy + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_5_miss <- lm(pandem ~ rule + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_6_miss <- lm(pandem ~ polyarchy + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)



### CASES PANBACK

model_panback_1_miss <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_2_miss <- lm(panback ~ rule + rule2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_3_miss <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_4_miss <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_5_miss <- lm(panback ~ rule + rule2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_6_miss <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)

In [35]:
diagnostic_tests(model_panback_6_miss)

VIF for collinearity of variables:

       Variables  Tolerance       VIF
1      polyarchy 0.03097424 32.284895
2     polyarchy2 0.02692779 37.136350
3    cases_ratio 0.70653374  1.415361
4    ethnic_frac 0.37350763  2.677321
5      ling_frac 0.41233126  2.425235
6     relig_frac 0.84078070  1.189371
7     gdp_pc_log 0.32219445  3.103716
8      inflation 0.88233539  1.133356
9  trade_gdp_log 0.84383701  1.185063
10           oil 0.65076664  1.536649
11       mineral 0.73047497  1.368972
12   density_log 0.72816696  1.373311



Kolmogorov-Smirnov for normality:

	One-sample Kolmogorov-Smirnov test

data:  y
D = 0.073098, p-value = 0.4294
alternative hypothesis: two-sided




RESET test for correct specification:

	RESET test

data:  model
RESET = 1.2691, df1 = 2, df2 = 128, p-value = 0.2846




Breusch-Pagan test for homogeneity:

	studentized Breusch-Pagan test

data:  model
BP = 16.277, df = 12, p-value = 0.1789




Durbin-Watson test for autocorrelation:

	Durbin-Watson test

data:  

In [36]:
robust_pandem_1_miss = coeftest(model_pandem_1_miss, vcov=vcovHC(model_pandem_1_miss, type="HC0"))
robust_pandem_3_miss = coeftest(model_pandem_3_miss, vcov=vcovHC(model_pandem_3_miss, type="HC0"))

robust_panback_1_miss = coeftest(model_panback_1_miss, vcov=vcovHC(model_panback_1_miss, type="HC0"))
robust_panback_2_miss = coeftest(model_panback_2_miss, vcov=vcovHC(model_panback_2_miss, type="HC0"))
robust_panback_3_miss = coeftest(model_panback_3_miss, vcov=vcovHC(model_panback_3_miss, type="HC0"))

In [37]:
# ## Summary

# stargazer(model_pandem_1, robust_pandem_1_miss,
#           model_pandem_2, model_pandem_2_miss,
#           model_pandem_3, robust_pandem_3_miss,
#           type="html", out="wo_military/MISS_pandem_FE.html")

# stargazer(model_pandem_4, model_pandem_4_miss,
#           model_pandem_5, model_pandem_5_miss,
#           model_pandem_6, model_pandem_6_miss,
#           type="html", out="wo_military/MISS_pandem_wFE.html")

# stargazer(model_panback_1, robust_panback_1_miss,
#           model_panback_2, robust_panback_2_miss,
#           model_panback_3, robust_panback_3_miss,
#           type="html", out="wo_military/MISS_panback_FE.html")

# stargazer(model_panback_4, model_panback_4_miss,
#           model_panback_5, model_panback_5_miss,
#           model_panback_6, model_panback_6_miss,
#           type="html", out="wo_military/MISS_panback_wFE.html")

In [38]:
# stargazer(model_pandem_1, robust_pandem_1_miss, robust_panback_1, robust_panback_1_miss, 
#           model_pandem_2, model_pandem_2_miss, robust_panback_2, robust_panback_2_miss,
#           model_pandem_3, robust_pandem_3_miss, robust_panback_3, robust_panback_3_miss,
#           type="html", out="wo_military/FINAL_MISS_FE.html")

# stargazer(model_pandem_4, model_pandem_4_miss, model_panback_4, model_panback_4_miss,
#           model_pandem_5, model_pandem_5_miss, model_panback_5, model_panback_5_miss,
#           model_pandem_6, model_pandem_6_miss, model_panback_6, model_panback_6_miss,
#           type="html", out="wo_military/FINAL_MISS_wFE.html")

In [39]:
stargazer(robust_panback_1_miss, robust_pandem_1_miss,  
          robust_panback_2_miss, model_pandem_2_miss, 
          robust_panback_3_miss, robust_pandem_3_miss, 
          type="html", out="../../results/robustness_check/robustness_MISS_FE.html")

stargazer(model_panback_4_miss, model_pandem_4_miss, 
          model_panback_5_miss, model_pandem_5_miss, 
          model_panback_6_miss, model_pandem_6_miss, 
          type="html", out="../../results/robustness_check/robustness_MISS_wFE.html")


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="3"></td><td>pandem</td><td colspan="2"></td></tr>
<tr><td style="text-align:left"></td><td colspan="3"><em>coefficient</em></td><td><em>OLS</em></td><td colspan="2"><em>coefficient</em></td></tr>
<tr><td style="text-align:left"></td><td colspan="3"><em>test</em></td><td><em></em></td><td colspan="2"><em>test</em></td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.575<sup>***</sup></td><td>-0.006</td><td>0.773<sup>***</sup></td><td>-0.158<sup>**</sup></td><td></td><td></td></tr>
<tr


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.478<sup>***</sup></td><td>-0.061</td><td>0.751<sup>***</sup></td><td>-0.198<sup>***</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.140)</td><td>(0.071)</td><td>(0.126)</td><td>(0.054)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text